## Import Pandas package

In [32]:
import pyspark.pandas as ps

path = "abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/bronze/Nationwide/"

## Function for listing all files in a group folder**

In [33]:
def deep_ls(path: str, max_depth=4):
    """
    List all files and folders in specified path and
    subfolders within maximum recursion depth.
    """

    # List all files in path
    li = mssparkutils.fs.ls(path)

    # Return all files
    for x in li:
        if x.size != 0:
            yield x

    # If the max_depth has not been reached, start
    # listing files and folders in subdirectories
    if max_depth > 1:
        for x in li:
            if x.size != 0:
                continue
            for y in deep_ls(x.path, max_depth - 1):
                yield y

    # If max_depth has been reached,
    # return the folders
    else:
        for x in li:
            if x.size == 0:
                yield x
 # If the max_depth has not been reached, start
    # listing files and folders in subdirectories
    if max_depth > 1:
        for x in li:
            if x.size != 0:
                continue
            for y in deep_ls(x.path, max_depth - 1):
                yield y

    # If max_depth has been reached,
    # return the folders
    else:
        for x in li:
            if x.size == 0:
                yield x


### Create a list of all excel files in bronze folder

In [34]:
files = []
files_generator = deep_ls(path)
for file in files_generator:
    files.append(file)

### Create a list of data frames that has data for each excel file

In [35]:
dataframe_list = []
for file in files:
    path_name = int(file.name.replace(".xlsx",""))
    spdf = ps.read_excel(file.path, skiprows =1)
    spdf["date_int"] = path_name
    dataframe_list.append(spdf)


### Collapse all tables into one data frame

In [36]:
spdfNationwideSilver = ps.concat(dataframe_list)
spdfNationwideSilver.reset_index(inplace = True)

### Rename columns with more appropriate names

In [37]:
new_names = ['idx', 'AgencyNumber', 'NotificationType', 'DateVal', 'Insured',
       'BillingAccountNumber', 'Policy', 'Status', 'CancelDate', 'Company',
       'PaymentAcceptanceDate', 'TotalBalance', 'MinimumDue',
       'NonSufficientFunds', 'DateReinstated', 'CancellationMailed',
       'dateint']
spdfNationwideSilver.columns = new_names

### Convert from pandas to spark data frame

In [38]:
sdfNationwideSilver = spdfNationwideSilver.to_spark()

### Save contents of data frame into Delta format

In [39]:
path = "abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/silver/nationwide"
try:
    mssparkutils.fs.rm(path, recurse=True)
except Exception as e:
    pass
sdfNationwideSilver.write.format("delta").option("overwrite","true").save(path)